In [1]:
import pandas as pd
from gensim import models, corpora
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords, stem_text, strip_non_alphanum, strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_short, strip_numeric
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import multiprocessing as mp
import nltk
import pandas as pd
import numpy as np
import tensorflow as tf
import parmap
import swifter

/home/ubuntu/anaconda3/envs/econ/lib/python3.7/site-packages/dask/dataframe/utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
topics_index_to_name_map = {
    0: 'Agriculture, animals, food and rural affairs',
    1: 'Asylum, immigration and nationality',
    2: 'Business, industry and consumers',
    3: 'Communities and families',
    4: 'Crime, civil law, justice and rights',
    5: 'Culture, media and sport',
    6: 'Defence',
    7: 'Economy and finance',
    8: 'Education',
    9: 'Employment and training',
    10: 'Energy and environment',
    11: 'European Union',
    12: 'Health services and medicine',
    13: 'Housing and planning',
    14: 'International affairs',
    15: 'Parliament, government and politics',
    16: 'Science and technology',
    17: 'Social security and pensions',
    18: 'Social services',
    19: 'Transport',
    20: 'Others'
}
topics_name_to_index_map = {y:x for x,y in topics_index_to_name_map.items()}

def strip_short2(text):
    return strip_short(text, minsize=4)


def preprocess_text(text):
    FILTERS = [lambda x: x.lower(), strip_multiple_whitespaces, strip_tags, strip_punctuation,
                   strip_non_alphanum, strip_numeric, strip_short2]
    return preprocess_string(text, FILTERS)

def preprocess(topic):
    ret = []
    topic = topic.strip()
    
    if '|' in topic:
        topics = topic.split('|')
        t = topics[0]
        t = t.strip()
        return topics_name_to_index_map[t]
        
    return topics_name_to_index_map[topic]

In [22]:
df = pd.read_csv('data/news_2016_predictions.csv')

In [23]:
df = df[df.month == 3]
df = df.reset_index()
df

,index,Unnamed: 0,source_id,source,day,month,year,program_name,transcript,parliament,top1_topic,top1_acc,top2_topic,top2_acc,top3_topic,top3_acc
0,318650,318650,163795,Belfast Telegraph,1,3,2016,CATTERICK,18+ Helpline Phonovation 0818 217 100 Going: G...,NaN,"Parliament, government and politics",22.93,Others,10.02,"Business, industry and consumers",8.48
1,318651,318651,163795,Belfast Telegraph,1,3,2016,LINGFIELD,18+ Helpline Phonovation 0818 217 100 CAPT KEE...,NaN,"Parliament, government and politics",18.24,"Culture, media and sport",14.50,"Business, industry and consumers",11.02
2,318652,318652,163795,Belfast Telegraph,1,3,2016,LEICESTER,18+ Helpline Phonovation 0818 217 100 Going: S...,NaN,"Business, industry and consumers",19.44,International affairs,13.14,"Parliament, government and politics",12.32
3,318653,318653,163795,Belfast Telegraph,1,3,2016,Ni Chuilin: Windsor perfect for next Carl fight,Sinn Fein's Caral Ni Chuilin appealed to promo...,NaN,"Culture, media and sport",60.87,"Parliament, government and politics",11.89,Transport,7.47
4,318654,318654,163795,Belfast Telegraph,1,3,2016,Judge in warning to Trust over records,Barney McElholm said that he would consider is...,NaN,"Crime, civil law, justice and rights",16.12,"Parliament, government and politics",15.08,Others,11.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167061,485711,485711,412338,Wales,31,3,2016,"Cruises to France, Spain and Portugal will sai...",Four departures have been announced for June 2...,NaN,Transport,63.43,European Union,10.99,"Parliament, government and politics",8.34
167062,485712,485712,412338,Wales,31,3,2016,"Cruises to France, Spain and Portugal will set...",Four departures have been announced for June 2...,NaN,Transport,60.48,European Union,10.94,Others,9.58
167063,485713,485713,412338,Wales,31,3,2016,"Ofcom tested EE, O2, Vodafone and Three for mo...","EE, O2, Three and Vodafone were compared when ...",NaN,"Culture, media and sport",34.59,Science and technology,18.17,Transport,14.92
167064,485714,485714,412338,Wales,31,3,2016,Reform business rates now to save the Welsh hi...,The Welsh Retail Consortium is calling on the ...,NaN,Economy and finance,34.99,"Parliament, government and politics",29.27,"Business, industry and consumers",10.44


In [26]:
df  = df.dropna(subset=['transcript'])

In [27]:
total_articles = len(df)
print('total number of news articles: {}'.format(total_articles))

total number of news articles: 167050


In [28]:
df['top1_topic'].value_counts()

Culture, media and sport                        41101
Others                                          35545
Parliament, government and politics             26722
Crime, civil law, justice and rights            12142
International affairs                           11736
Business, industry and consumers                 9590
Transport                                        6808
Health services and medicine                     5430
Economy and finance                              4563
European Union                                   3669
Agriculture, animals, food and rural affairs     3613
Defence                                          1766
Education                                        1498
Energy and environment                           1024
Employment and training                           994
Communities and families                          562
Asylum, immigration and nationality               171
Housing and planning                              114
Social security and pensions

In [29]:
df = df[df.top1_acc >= 0.4]
df = df[df.top1_topic != 'Others']

In [30]:
df['top1_topic'].value_counts()

Culture, media and sport                        41101
Parliament, government and politics             26722
Crime, civil law, justice and rights            12142
International affairs                           11736
Business, industry and consumers                 9590
Transport                                        6808
Health services and medicine                     5430
Economy and finance                              4563
European Union                                   3669
Agriculture, animals, food and rural affairs     3613
Defence                                          1766
Education                                        1498
Energy and environment                           1024
Employment and training                           994
Communities and families                          562
Asylum, immigration and nationality               171
Housing and planning                              114
Social security and pensions                        2
Name: top1_topic, dtype: int

In [31]:
df = df.drop(['Unnamed: 0'], axis=1)

In [32]:
print('articles after drop: {}'.format(len(df)))

articles after drop: 131505


In [33]:
print('percent of articles remaining {}'.format(len(df)/total_articles))

percent of articles remaining 0.7872193953906016


In [34]:
X = df['transcript'].values
Y = df['top1_topic'].values

In [35]:
print('preprocessing data!!')
preprocessed_X = parmap.map(preprocess_text, X, pm_pbar=True)
tagged_X = [models.doc2vec.TaggedDocument(doc, [i]) for i, doc in enumerate(preprocessed_X)]

preprocessing data!!


131584it [00:18, 7073.21it/s]                            


In [36]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
input_Y = Y.reshape(-1,1)
enc.fit(input_Y)
input_Y = enc.transform(input_Y).toarray()

In [37]:
enc.categories_

[array(['Agriculture, animals, food and rural affairs',
        'Asylum, immigration and nationality',
        'Business, industry and consumers', 'Communities and families',
        'Crime, civil law, justice and rights', 'Culture, media and sport',
        'Defence', 'Economy and finance', 'Education',
        'Employment and training', 'Energy and environment',
        'European Union', 'Health services and medicine',
        'Housing and planning', 'International affairs',
        'Parliament, government and politics',
        'Social security and pensions', 'Transport'], dtype=object)]

In [38]:
print('training doc2vec')
doc2vec_model = Doc2Vec(vector_size=100, window=3, workers=mp.cpu_count(), epochs=40)
doc2vec_model.build_vocab(tagged_X)
doc2vec_model.train(tagged_X, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)
doc2vec_model.save('./models/doc2vec/doc2vec_news_2016')

training doc2vec


In [39]:
print('preparing inputs')

def get_doc_vec(doc):
    return doc2vec_model.infer_vector(doc.words)

inputs = parmap.map(get_doc_vec, tagged_X, pm_pbar=True)

# for x in tagged_X:
#     topic_vec = doc2vec_model.infer_vector(x.words)
#     inputs.append(topic_vec)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs, input_Y, test_size=0.2, random_state=42)
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
print('X_train: ', X_train.shape)
print('y_train: ', y_train.shape)
print('X_test: ', X_test.shape)
print('y_test: ', y_test.shape)

preparing inputs


131584it [03:35, 610.72it/s]                             


X_train:  (105204, 100)
y_train:  (105204, 18)
X_test:  (26301, 100)
y_test:  (26301, 18)


In [104]:
def build_network():
    model = tf.keras.models.Sequential([
        tf.keras.Input(shape=(100)),
        tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.1)),
        tf.keras.layers.Dense(21, activation='softmax')
    ])
    loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False, label_smoothing=0, name='categorical_crossentropy')

    model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=loss_fn,
              metrics=['accuracy'])
    
    return model

In [105]:
model = build_network()
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_train, y_train, batch_size=32, epochs=200, callbacks=[callback], validation_data=(X_test, y_test))

Train on 47439 samples, validate on 11860 samples
Epoch 1/200
47439/47439 [==============================] - 9s 193us/sample - loss: 1.7034 - accuracy: 0.6442 - val_loss: 1.1368 - val_accuracy: 0.6992
Epoch 2/200
47439/47439 [==============================] - 7s 144us/sample - loss: 1.0703 - accuracy: 0.7110 - val_loss: 1.0319 - val_accuracy: 0.7122
Epoch 3/200
47439/47439 [==============================] - 6s 122us/sample - loss: 0.9982 - accuracy: 0.7174 - val_loss: 0.9851 - val_accuracy: 0.7134
Epoch 4/200
47439/47439 [==============================] - 9s 187us/sample - loss: 0.9646 - accuracy: 0.7191 - val_loss: 0.9583 - val_accuracy: 0.7132
Epoch 5/200
47439/47439 [==============================] - 5s 103us/sample - loss: 0.9434 - accuracy: 0.7220 - val_loss: 0.9321 - val_accuracy: 0.7197
Epoch 6/200
47439/47439 [==============================] - 4s 82us/sample - loss: 0.9279 - accuracy: 0.7229 - val_loss: 0.9270 - val_accuracy: 0.7152
Epoch 7/200
47439/47439 [====================

In [106]:
model.save_weights('./models/doc2vec/doc2vec_news_classifier')

In [127]:
def predict(X, topn=3):
    preprocessed_X = preprocess_text(X)
    vec_X = doc2vec_model.infer_vector(preprocessed_X)
    vec_X = vec_X.reshape(1, vec_X.shape[0])
    pred = model.predict(vec_X)
    pred_i = [(topics_index_to_name_map[i], p) for i,p in enumerate(pred[0])]
    pred_sorted = sorted(pred_i, key=lambda x: x[1], reverse=True)
    return pred_sorted[:topn]

In [117]:
df

,source_id,source,day,month,year,program_name,transcript,predicted_topic,top_class,topic
0,163795,Belfast Telegraph,1,1,2014,Hunt begins for team to secure cultural legacy,"As the clock struck midnight last night, Londo...","[(Science and technology, 0.7244457), (Culture...",Science and technology,16
3,163795,Belfast Telegraph,1,1,2014,"Derry treated as poor relation, say rail campa...",Campaigners from Into The West argued that Der...,"[(Transport, 0.6714804), (Parliament, governme...",Transport,19
5,163795,Belfast Telegraph,1,1,2014,FAILURE REVERBERATES;\nFrustration in Washingt...,Minutes before the final text of the negotiati...,"[(Parliament, government and politics, 0.55315...","Parliament, government and politics",15
6,163795,Belfast Telegraph,1,1,2014,5 GREAT THINGS TO DO,Little 'uns can get right up close to the acti...,"[(Culture, media and sport, 0.46135047), (Othe...","Culture, media and sport",5
9,163795,Belfast Telegraph,1,1,2014,"Despite a lot of seemingly tireless work, it g...",Many had hung around - seemingly unwilling or ...,"[(Parliament, government and politics, 0.40623...","Parliament, government and politics",15
...,...,...,...,...,...,...,...,...,...,...
147277,412338,Wales,31,1,2014,Six Nations 2014: 26 pictures of the great pla...,Since the Grand Slam win in 2005 Wales have wo...,"[(Parliament, government and politics, 0.84473...","Parliament, government and politics",15
147278,412338,Wales,31,1,2014,"Aled Blake: Rejoice, rugby's cultural monopoly...",An overstatement? I don't think so. Why? At th...,"[(Parliament, government and politics, 0.58371...","Parliament, government and politics",15
147279,412338,Wales,31,1,2014,Wales weather: Fresh storms threaten more floo...,"A combination of driving rain, high tides, a p...","[(Parliament, government and politics, 0.44981...","Parliament, government and politics",15
147280,412338,Wales,31,1,2014,Wales weather: Weekend high tides expected to ...,"A combination of driving rain, high tides, a p...","[(Parliament, government and politics, 0.55013...","Parliament, government and politics",15


In [118]:
df['predicted_topic2'] = df.swifter.apply(lambda row: predict(row['transcript']), axis=1)

(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)

(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)

(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)

(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)

(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)

...

(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)

(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)

(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)

(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)

(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)

In [119]:
df

,source_id,source,day,month,year,program_name,transcript,predicted_topic,top_class,topic,predicted_topic2
0,163795,Belfast Telegraph,1,1,2014,Hunt begins for team to secure cultural legacy,"As the clock struck midnight last night, Londo...","[(Science and technology, 0.7244457), (Culture...",Science and technology,16,"[(Parliament, government and politics, 0.40627..."
3,163795,Belfast Telegraph,1,1,2014,"Derry treated as poor relation, say rail campa...",Campaigners from Into The West argued that Der...,"[(Transport, 0.6714804), (Parliament, governme...",Transport,19,"[(Transport, 0.9525657), (Parliament, governme..."
5,163795,Belfast Telegraph,1,1,2014,FAILURE REVERBERATES;\nFrustration in Washingt...,Minutes before the final text of the negotiati...,"[(Parliament, government and politics, 0.55315...","Parliament, government and politics",15,"[(Parliament, government and politics, 0.84827..."
6,163795,Belfast Telegraph,1,1,2014,5 GREAT THINGS TO DO,Little 'uns can get right up close to the acti...,"[(Culture, media and sport, 0.46135047), (Othe...","Culture, media and sport",5,"[(Culture, media and sport, 0.6547228), (Parli..."
9,163795,Belfast Telegraph,1,1,2014,"Despite a lot of seemingly tireless work, it g...",Many had hung around - seemingly unwilling or ...,"[(Parliament, government and politics, 0.40623...","Parliament, government and politics",15,"[(Parliament, government and politics, 0.61372..."
...,...,...,...,...,...,...,...,...,...,...,...
147277,412338,Wales,31,1,2014,Six Nations 2014: 26 pictures of the great pla...,Since the Grand Slam win in 2005 Wales have wo...,"[(Parliament, government and politics, 0.84473...","Parliament, government and politics",15,"[(Culture, media and sport, 0.7159495), (Parli..."
147278,412338,Wales,31,1,2014,"Aled Blake: Rejoice, rugby's cultural monopoly...",An overstatement? I don't think so. Why? At th...,"[(Parliament, government and politics, 0.58371...","Parliament, government and politics",15,"[(Parliament, government and politics, 0.88941..."
147279,412338,Wales,31,1,2014,Wales weather: Fresh storms threaten more floo...,"A combination of driving rain, high tides, a p...","[(Parliament, government and politics, 0.44981...","Parliament, government and politics",15,"[(Energy and environment, 0.5894057), (Parliam..."
147280,412338,Wales,31,1,2014,Wales weather: Weekend high tides expected to ...,"A combination of driving rain, high tides, a p...","[(Parliament, government and politics, 0.55013...","Parliament, government and politics",15,"[(Energy and environment, 0.6159892), (Parliam..."


In [121]:
df = df.drop(['top_class', 'topic'], axis=1)

In [122]:
df.to_csv('./doc2vec_on_news_transcripts_preds.csv')

In [123]:
import os
path = './data/bbc/2016/transcripts'
files = os.listdir(path)

data = []
for source in files:
    data.append(pd.read_csv(os.path.join(path, source)))
df = pd.concat(data)
df = df.drop(['Unnamed: 0', 'Has Transcript', 'Unavailable link', 'Unavailable reason'], axis=1)
df

,Source,Date,Program Name,Time,Duration,Transcript
0,BBC1 London,6-dec-2014,BBC News,22:00,20 mins,"#Ah... # Dreaming ofthe days # APPLAUSE Hiya, ..."
1,BBC1 London,7-dec-2014,BBC News,22:00,20 mins,Alex Salmond unveils plans to seek a Westminst...
2,BBC1 London,7-dec-2014,BBC London News,22:20,10 mins,against the illegal trade in wildlife. He’s ve...
3,BBC1 London,5-dec-2014,BBC News at One,13:00,30 mins,# The way I love you. # Making Christmas speci...
4,BBC1 London,5-dec-2014,BBC News at Ten,22:00,25 mins,Two British men are jailed for travelling to S...
...,...,...,...,...,...,...
107,BBC1 London,26-jun-2014,BBC News at One,13:00,30 mins,"Hello, Glastonbury. Hello, Glastonbury! Hello,..."
108,BBC1 London,26-jun-2014,BBC London News,13:30,15 mins,A little bit more cloud around but some of us ...
109,BBC1 London,26-jun-2014,BBC News at Six,19:10,25 mins,Yet more shocking re-lations about the extent ...
110,BBC1 London,26-jun-2014,BBC News at Ten,22:00,30 mins,The most detailed picture yet ofthe The most d...


In [124]:
df['predicted_topic'] = df.swifter.apply(lambda row: predict(row['Transcript']), axis=1)

/home/ubuntu/anaconda3/envs/econ/lib/python3.7/site-packages/swifter/swifter.py:55: UserWarning: This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`.
  "This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`."


(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)

(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)



In [125]:
df

,Source,Date,Program Name,Time,Duration,Transcript,predicted_topic
0,BBC1 London,6-dec-2014,BBC News,22:00,20 mins,"#Ah... # Dreaming ofthe days # APPLAUSE Hiya, ...","[(Parliament, government and politics, 0.37201..."
1,BBC1 London,7-dec-2014,BBC News,22:00,20 mins,Alex Salmond unveils plans to seek a Westminst...,"[(Parliament, government and politics, 0.70019..."
2,BBC1 London,7-dec-2014,BBC London News,22:20,10 mins,against the illegal trade in wildlife. He’s ve...,"[(Transport, 0.40607873), (Culture, media and ..."
3,BBC1 London,5-dec-2014,BBC News at One,13:00,30 mins,# The way I love you. # Making Christmas speci...,"[(Parliament, government and politics, 0.42028..."
4,BBC1 London,5-dec-2014,BBC News at Ten,22:00,25 mins,Two British men are jailed for travelling to S...,"[(Crime, civil law, justice and rights, 0.5460..."
...,...,...,...,...,...,...,...
107,BBC1 London,26-jun-2014,BBC News at One,13:00,30 mins,"Hello, Glastonbury. Hello, Glastonbury! Hello,...","[(Crime, civil law, justice and rights, 0.6193..."
108,BBC1 London,26-jun-2014,BBC London News,13:30,15 mins,A little bit more cloud around but some of us ...,"[(Culture, media and sport, 0.40613863), (Parl..."
109,BBC1 London,26-jun-2014,BBC News at Six,19:10,25 mins,Yet more shocking re-lations about the extent ...,"[(Crime, civil law, justice and rights, 0.5205..."
110,BBC1 London,26-jun-2014,BBC News at Ten,22:00,30 mins,The most detailed picture yet ofthe The most d...,"[(Crime, civil law, justice and rights, 0.5720..."


In [126]:
df.to_csv('doc2vec_on_news_bbc_2016_preds.csv')